In [1]:
from dataset import SiameseGoogleFer

In [2]:
train_path = "data/faceexp-comparison-data-train-public.csv"
test_path = "data/faceexp-comparison-data-test-public.csv"
train_dataset = SiameseGoogleFer(train_path, train_flag=True)
#test_dataset = SiameseGoogleFer(test_path, train_flag=False)

AttributeError: 'NoneType' object has no attribute 'shape'